In [ ]:
import uproot
import ROOT

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('bmh')

In [ ]:
# tree = uproot.open('../Firefighter/washAOD/test/SIDMmumu_Mps-200_MZp-1p2_ctau-1p2_100k.root')['RECO_muTrackEffi/recoEffiForMuTrack']
# df = tree.pandas.df()

from collections import OrderedDict

dfs = OrderedDict({
    0.1 : uproot.open('../Firefighter/washAOD/test/SIDMmumu_Mps-200_MZp-1p2_ctau-1p20e-03_100k.root')['RECO_muTrackEffi/recoEffiForMuTrack'].pandas.df(),
      1 : uproot.open('../Firefighter/washAOD/test/SIDMmumu_Mps-200_MZp-1p2_ctau-0p012_100k.root')['RECO_muTrackEffi/recoEffiForMuTrack']   .pandas.df(),
     10 : uproot.open('../Firefighter/washAOD/test/SIDMmumu_Mps-200_MZp-1p2_ctau-0p12_100k.root')['RECO_muTrackEffi/recoEffiForMuTrack']    .pandas.df(),
     50 : uproot.open('../Firefighter/washAOD/test/SIDMmumu_Mps-200_MZp-1p2_ctau-0p6_100k.root')['RECO_muTrackEffi/recoEffiForMuTrack']     .pandas.df(),
    100 : uproot.open('../Firefighter/washAOD/test/SIDMmumu_Mps-200_MZp-1p2_ctau-1p2_100k.root')['RECO_muTrackEffi/recoEffiForMuTrack']     .pandas.df(),
    300 : uproot.open('../Firefighter/washAOD/test/SIDMmumu_Mps-200_MZp-1p2_ctau-3p6_100k.root')['RECO_muTrackEffi/recoEffiForMuTrack']     .pandas.df()
})

In [ ]:
for k, df in dfs.items():
    df.columns = [x.decode() for x in df.columns]

In [ ]:
fig, ax = plt.subplots()
plt.rcParams['patch.linewidth'] = 1.2

for k, df in dfs.items():
    df['nMatched'].plot.hist(bins=5, range=(0,5), ax=ax, label='%.2fcm'%k, histtype='step', density=True)
    print('%.2fcm #events: %i' % (k, len(df['nMatched'].dropna())))
ax.set_xlabel('#muons')
plt.title('reconstructed offline muon(dSA) per event')
plt.legend(loc='upper left', title='decay length')

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(15,4))

df = dfs[100]

recoDf = df[['genPt','genEta','genPhi', 'recoPt']].dropna()

df['genPt']    .plot.hist(bins=50, range=(0,140), ax=ax[0], label='all')
recoDf['genPt'].plot.hist(bins=50, range=(0,140), ax=ax[0], label='recoed')
ax[0].set_xlabel('pT [GeV]')

df['genEta']    .plot.hist(bins=50, range=(-2.4,2.4), ax=ax[1], label='all')
recoDf['genEta'].plot.hist(bins=50, range=(-2.4,2.4), ax=ax[1], label='recoed')
ax[1].set_xlabel('eta')

df['genPhi']    .plot.hist(bins=50, ax=ax[2], label='all')
recoDf['genPhi'].plot.hist(bins=50, ax=ax[2], label='recoed')
ax[2].set_xlabel('phi')

for a in ax: a.legend(loc='best')

In [ ]:

_c = ROOT.TCanvas()

h0s = OrderedDict()
h1s = OrderedDict()
effs = OrderedDict()
effgs = OrderedDict()

for k, df in dfs.items():
    h0s[k] = ROOT.TH1F('h0_%.2f'%k, '',50,0,140)
    for x in df['genPt']: h0s[k].Fill(x)
    h1s[k] = ROOT.TH1F('h1_%.2f'%k, '',50,0,140)
    for x in recoDf['genPt']: h1s[k].Fill(x)

    effs[k] = ROOT.TEfficiency(h1s[k], h0s[k])
    effs[k].SetTitle('muon reconstruction efficiency;p_{T} [GeV];Efficiency')

    effs[k].Draw()
    ROOT.gPad.Update()
    effgs[k] = effs[k].GetPaintedGraph()

c = ROOT.TCanvas()
c.SetGrid(1)
colors = [ROOT.kRed, ROOT.kGreen, ROOT.kBlue, ROOT.kYellow, ROOT.kMagenta, ROOT.kCyan]
leg = ROOT.TLegend(0.55,0.1,0.75,0.5)

for i, k in enumerate(effgs):
    effgs[k].GetYaxis().SetRangeUser(0,1)
    effgs[k].SetMarkerStyle(ROOT.kCircle)
    effgs[k].SetMarkerSize(0.5)
    effgs[k].SetMarkerColor(colors[i])
    effgs[k].SetLineColor(colors[i])
    effgs[k].Draw('AP' if i==0 else 'P')
    leg.AddEntry(effgs[k], '%.2fcm'%k, 'lep')

leg.Draw()
c.Draw()

In [ ]:
_c = ROOT.TCanvas()

h0s = OrderedDict()
h1s = OrderedDict()
effs = OrderedDict()
effgs = OrderedDict()

for k, df in dfs.items():
    h0s[k] = ROOT.TH1F('h0_%.2f'%k, '',50,-2.4,2.4)
    for x in df['genEta']: h0s[k].Fill(x)
    h1s[k] = ROOT.TH1F('h1_%.2f'%k, '',50,-2.4,2.4)
    for x in recoDf['genEta']: h1s[k].Fill(x)

    effs[k] = ROOT.TEfficiency(h1s[k], h0s[k])
    effs[k].SetTitle('muon reconstruction efficiency;#eta;Efficiency')

    effs[k].Draw()
    ROOT.gPad.Update()
    effgs[k] = effs[k].GetPaintedGraph()

c = ROOT.TCanvas()
c.SetGrid(1)
colors = [ROOT.kRed, ROOT.kGreen, ROOT.kBlue, ROOT.kYellow, ROOT.kMagenta, ROOT.kCyan]
leg = ROOT.TLegend(0.55,0.1,0.75,0.5)

for i, k in enumerate(effgs):
    effgs[k].GetYaxis().SetRangeUser(0,1)
    effgs[k].SetMarkerStyle(ROOT.kCircle)
    effgs[k].SetMarkerSize(0.5)
    effgs[k].SetMarkerColor(colors[i])
    effgs[k].SetLineColor(colors[i])
    effgs[k].Draw('AP' if i==0 else 'P')
    leg.AddEntry(effgs[k], '%.2fcm'%k, 'lep')

leg.Draw()
c.Draw()